<a href="https://colab.research.google.com/github/sravansai04/sravansai04-Hacker-Earthh-Ml-Challange-Exhibit-A-rt/blob/main/Predict_the_cost_to_ship_the_sculptures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip /content/drive/MyDrive/predict.zip

Archive:  /content/drive/MyDrive/predict.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/test.csv        
  inflating: dataset/sample_submission.csv  


# Import Data-Sets











In [ ]:
train=pd.read_csv("/content/dataset/train.csv")
train.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777",-283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072",-159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241",-154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375",-161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793",-159.23


In [ ]:
test=pd.read_csv("/content/dataset/test.csv")
test.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location
0,fffe3400310033003300,James Miller,0.35,53.0,18.0,871.0,Wood,5.98,19.11,Yes,Yes,No,Airways,No,Working Class,No,07/03/17,07/06/17,"Santoshaven, IA 63481"
1,fffe3600350035003400,Karen Vetrano,0.67,7.0,4.0,108.0,Clay,6.92,13.96,No,No,No,Roadways,Yes,Working Class,No,05/02/16,05/02/16,"Ericksonton, OH 98253"
2,fffe3700360030003500,Roseanne Gaona,0.61,6.0,5.0,97.0,Aluminium,4.23,13.62,Yes,No,No,Airways,No,Working Class,No,01/04/18,01/06/18,APO AP 83453
3,fffe350038003600,Todd Almanza,0.14,15.0,8.0,757.0,Clay,6.28,23.79,No,Yes,No,Roadways,Yes,Wealthy,No,09/14/17,09/17/17,"Antonioborough, AL 54778"
4,fffe3500390032003500,Francis Rivero,0.63,10.0,4.0,1673.0,Marble,4.39,17.83,No,Yes,Yes,Roadways,No,Working Class,Yes,12/03/17,12/02/17,"Lake Frances, LA 03040"


# Feature Engineering

## Checking Null Values

In [ ]:
train.shape

(6500, 20)

In [ ]:
test.shape

(3500, 19)

In [ ]:
train.isnull().sum()

Customer Id                 0
Artist Name                 0
Artist Reputation         750
Height                    375
Width                     584
Weight                    587
Material                  764
Price Of Sculpture          0
Base Shipping Price         0
International               0
Express Shipment            0
Installation Included       0
Transport                1392
Fragile                     0
Customer Information        0
Remote Location           771
Scheduled Date              0
Delivery Date               0
Customer Location           0
Cost                        0
dtype: int64

In [ ]:
test.isnull().sum()

Customer Id                0
Artist Name                0
Artist Reputation        222
Height                   119
Width                    141
Weight                   149
Material                   0
Price Of Sculpture         0
Base Shipping Price        0
International              0
Express Shipment           0
Installation Included      0
Transport                232
Fragile                    0
Customer Information       0
Remote Location            0
Scheduled Date             0
Delivery Date              0
Customer Location          0
dtype: int64

In [ ]:
train.dtypes

Customer Id               object
Artist Name               object
Artist Reputation        float64
Height                   float64
Width                    float64
Weight                   float64
Material                  object
Price Of Sculpture       float64
Base Shipping Price      float64
International             object
Express Shipment          object
Installation Included     object
Transport                 object
Fragile                   object
Customer Information      object
Remote Location           object
Scheduled Date            object
Delivery Date             object
Customer Location         object
Cost                     float64
dtype: object

In [ ]:
test.dtypes

Customer Id               object
Artist Name               object
Artist Reputation        float64
Height                   float64
Width                    float64
Weight                   float64
Material                  object
Price Of Sculpture       float64
Base Shipping Price      float64
International             object
Express Shipment          object
Installation Included     object
Transport                 object
Fragile                   object
Customer Information      object
Remote Location           object
Scheduled Date            object
Delivery Date             object
Customer Location         object
dtype: object

In [ ]:
train['Material'].value_counts()

Brass        847
Aluminium    845
Bronze       821
Marble       819
Wood         816
Clay         816
Stone        772
Name: Material, dtype: int64

In [ ]:
train['Transport'].value_counts()

Roadways     2064
Airways      1817
Waterways    1227
Name: Transport, dtype: int64

In [ ]:
train['Customer Information'].value_counts()

Working Class    4803
Wealthy          1697
Name: Customer Information, dtype: int64

In [ ]:
#all the customers are staying in different locations so now we need to map these things
len(train['Customer Location'].unique())


6500

In [ ]:
train['Express Shipment'].value_counts()

No     4365
Yes    2135
Name: Express Shipment, dtype: int64

In [ ]:
train['Fragile'].value_counts()

No     5461
Yes    1039
Name: Fragile, dtype: int64

In [ ]:
train_Y = train['Cost'].abs()
train_X = train.drop(['Cost'], axis=1)
test_X = test
train_X.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777"
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072"
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241"
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375"
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793"


# Feature Selection

In [ ]:
train_X = train_X.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location', 'Transport'], axis = 1)
test_X = test.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location', 'Transport'], axis = 1)

In [ ]:
train_X.dtypes

Artist Reputation        float64
Height                   float64
Width                    float64
Weight                   float64
Material                  object
Price Of Sculpture       float64
Base Shipping Price      float64
International             object
Express Shipment          object
Installation Included     object
Fragile                   object
Customer Information      object
Remote Location           object
dtype: object

# Filling Null Values

In [ ]:
def impute(DataFrame,ColName):
     most=DataFrame[ColName].mode()[0]
     DataFrame[ColName + "_Imputed"] = DataFrame[ColName]
     DataFrame[ColName + "_Imputed"].fillna(most,inplace=True)

for Columns in ['Material', 'Remote Location']:
    impute(train_X,Columns)
    impute(test_X,Columns)
    
train_X[['Material','Material_Imputed','Remote Location','Remote Location_Imputed']].head(10)
train_X = train_X.drop(['Material', 'Remote Location'], axis = 1)
test_X = test_X.drop(['Material', 'Remote Location'], axis = 1)


In [ ]:
train_X.head()

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Fragile,Customer Information,Material_Imputed,Remote Location_Imputed
0,0.26,17.0,6.0,4128.0,13.91,16.27,Yes,Yes,No,No,Working Class,Brass,No
1,0.28,3.0,3.0,61.0,6.83,15.00,No,No,No,No,Working Class,Brass,No
2,0.07,8.0,5.0,237.0,4.96,21.18,No,No,No,Yes,Working Class,Clay,Yes
3,0.12,9.0,NaN,NaN,5.81,16.31,No,No,No,No,Wealthy,Aluminium,Yes
4,0.15,17.0,6.0,324.0,3.18,11.94,Yes,Yes,Yes,No,Working Class,Aluminium,No


# Handling Categorical Features

In [ ]:
# Get list of categorical variables
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_X_train = train_X.copy()
label_X_test = test_X.copy()
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(train_X[col])
    label_X_test[col] = label_encoder.transform(test_X[col])

In [ ]:
label_X_train.isnull().sum()

Artist Reputation          750
Height                     375
Width                      584
Weight                     587
Price Of Sculpture           0
Base Shipping Price          0
International                0
Express Shipment             0
Installation Included        0
Fragile                      0
Customer Information         0
Material_Imputed             0
Remote Location_Imputed      0
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(label_X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(label_X_test))
imputed_X_train.columns = train_X.columns
imputed_X_test.columns = test_X.columns

# Scaling

In [ ]:
import sklearn
scaler = sklearn.preprocessing.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(imputed_X_train), columns=imputed_X_train.columns)
X_test = pd.DataFrame(scaler.transform(imputed_X_test), columns=imputed_X_test.columns)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, train_Y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
y_prediction_rf = rf.predict(X_test)
y_prediction_rf

array([   268.3527,    296.8799,    214.3183, ...,   1078.8713,
          290.9305, 150666.6508])

In [ ]:
random=pd.DataFrame()
random['Customer Id'] = test['Customer Id']
random['Cost'] = y_prediction_rf
random['Cost'] = random['Cost'].abs()
random.to_csv('random.csv', index = False)
#accuracy 90

## Hyper Parameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = [
{'n_estimators': [40,70,100,250,500], 
 'max_depth': [10,30,50,100], 'bootstrap': [True, False]}
]


In [ ]:
rsf=RandomizedSearchCV(rf, param_grid, cv=10)
rsf.fit(X_train, train_Y)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                             

In [ ]:
best_random=rsf.best_estimator_
best_random

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
best_random.fit(X_train, train_Y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
best_pred=best_random.predict(X_test)
best_pred

array([   208.16129626,    298.34138504,    258.86857607, ...,
          635.70962421,    275.86632867, 141683.87839547])

In [ ]:
trandom=pd.DataFrame()
trandom['Customer Id'] = test['Customer Id']
trandom['Cost'] = best_pred
trandom['Cost'] = trandom['Cost'].abs()
trandom.to_csv('trandom.csv', index = False)
#accuracy 85

# Gradient Bossting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor


In [ ]:
gbr= GradientBoostingRegressor(n_estimators=100, random_state=42)
gbr.fit(X_train, train_Y)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
gbr_prediction=gbr.predict(X_test)
gbr_prediction

array([  738.6276376 ,   437.12951029,   413.72433984, ...,
         480.3808884 ,   653.20246282, 80856.2201224 ])

In [ ]:
gradient=pd.DataFrame()
gradient['Customer Id'] = test['Customer Id']
gradient['Cost'] = gbr_prediction
gradient['Cost'] = gradient['Cost'].abs()
gradient.to_csv('gradient.csv', index = False)